In [31]:
!python merge_2_images.py --video_path=Videos/video.mp4 --overlay_image=ruler.png --output_image=merged_images.png

Saved the output to 'merged_images.png'


In [32]:
!python process_ratio_cm_per_pixles.py --input_image=merged_images.png --output_json=ratio.json

Start: (145, 121), End: (1146, 121)
area_for_segment: [145, 121, 1146, 121]
cm_per_pixel: 0.00999000999000999
Saved to 'ratio.json'


In [33]:
!python process_area_for_segmet.py --video_path=Videos/video.mp4 --output_json=area.json

area_for_segment = [1295, 1, 2338, 2154]
Saved to area.json


In [34]:
!python inference.py --video_path=Videos/video.mp4  --model_object_detection=Models/best.pt  --model_image_segmentation=Models/best_yoloseg.pt --output_path=inference_output/output.mp4 --output_data_path=output_data.json --area_segment_path=area.json --ratio_path=ratio.json --tracker_config=bytetrack_custom.yaml --conf_threshold=0.4

Processing Frames: 100%|███████████████████| 231/231 [00:46<00:00,  5.02frame/s]
Processed video saved to: inference_output/output.mp4
